# CrystalMath Quick Start Guide

This notebook demonstrates the basic usage of CrystalMath for submitting and monitoring DFT calculations.

You will learn how to:
1. Import CrystalController and models
2. Initialize the controller with backend selection
3. Create a simple CRYSTAL input
4. Submit a job using JobSubmission
5. Monitor job status
6. View job logs and results
7. Submit a VASP job

**Prerequisites:**
- CrystalMath installed: `uv sync` in the crystalmath repo
- Working directory: crystalmath/ root

## 1. Import CrystalMath

The main entry point is `CrystalController` from `crystalmath.api`.
We also import the core data models from `crystalmath.models`.

In [ ]:
from crystalmath.api import CrystalController
from crystalmath.models import (
    JobSubmission,
    JobStatus,
    JobDetails,
    DftCode,
    RunnerType,
    JobState,
)

print("CrystalMath imported successfully!")

## 2. Initialize the Controller

The `CrystalController` automatically selects the best available backend:
1. **AiiDA** (if installed and configured) - Best for production workflows
2. **SQLite** (if db_path provided) - Simple local database
3. **Demo** backend (fallback) - In-memory storage for testing

For this tutorial, we'll use SQLite for simplicity.

In [ ]:
# Initialize with SQLite backend (simplest for getting started)
ctrl = CrystalController(db_path="quickstart.db", use_aiida=False)

# Or use AiiDA backend (if configured):
# ctrl = CrystalController(use_aiida=True, profile_name="default")

print(f"Controller initialized with backend: {ctrl._backend.name}")

## 3. Create a CRYSTAL Input

Let's create a simple MgO SCF calculation input.
This is a standard CRYSTAL23 `.d12` format input file.

In [ ]:
# A simple MgO SCF calculation input
mgo_input = """MgO SCF calculation
CRYSTAL
0 0 0
225
4.21
2
12  0.0  0.0  0.0
8   0.5  0.5  0.5
END
12 3
8 3
END
SHRINK
8 8
TOLDEE
8
END
END
"""

print("Input prepared:")
print(mgo_input[:200] + "...")

## 4. Submit a Job

Use the `JobSubmission` model to define the job parameters.
The controller's `submit_job()` method returns a primary key (PK) for tracking.

In [ ]:
# Create job submission
job = JobSubmission(
    name="mgo_scf",
    dft_code=DftCode.CRYSTAL,
    input_content=mgo_input,
    runner_type=RunnerType.LOCAL,
)

# Submit to backend
pk = ctrl.submit_job(job)
print(f"Job submitted with PK: {pk}")

## 5. Monitor Job Status

Query the job list and check status using the PK.
The `get_jobs()` method returns a list of `JobStatus` models.

In [ ]:
# List all jobs
jobs = ctrl.get_jobs(limit=10)

print("Jobs in database:")
for j in jobs:
    print(f"  PK={j.pk} | {j.name:20s} | {j.state.value:12s} | {j.dft_code.value}")

In [ ]:
# Get detailed status for our specific job
details = ctrl.get_job_details(pk)

if details:
    print(f"\nJob Details:")
    print(f"  Name: {details.name}")
    print(f"  State: {details.state.value}")
    print(f"  DFT Code: {details.dft_code.value}")
    print(f"  Convergence: {details.convergence_met}")
    
    if details.final_energy:
        print(f"  Final Energy: {details.final_energy:.6f} Ha")
    
    if details.bandgap_ev is not None:
        print(f"  Band Gap: {details.bandgap_ev:.3f} eV")
    
    if details.warnings:
        print(f"  Warnings: {len(details.warnings)}")
        for w in details.warnings[:3]:
            print(f"    - {w}")

## 6. View Job Logs

The `get_job_log()` method returns stdout/stderr from the calculation.
Use the `tail_lines` parameter to limit output.

In [ ]:
# Get job logs (last 20 lines)
log = ctrl.get_job_log(pk, tail_lines=20)

if log.get("stdout"):
    print("=== STDOUT (last 10 lines) ===")
    for line in log["stdout"][-10:]:
        print(line)

if log.get("stderr"):
    print("\n=== STDERR ===")
    for line in log["stderr"][-5:]:
        print(line)

## 7. VASP Job Example

CrystalMath also supports VASP calculations.
You can submit VASP jobs using the same interface.

For real VASP calculations, see notebook `02_materials_project.ipynb` which shows
how to generate complete VASP inputs from Materials Project structures.

In [ ]:
# Example VASP job submission
vasp_job = JobSubmission(
    name="si_relax",
    dft_code=DftCode.VASP,
    runner_type=RunnerType.LOCAL,
    parameters={"preset": "relax"},
    input_content="Si\n4.2\n0 0.5 0.5\n...",  # POSCAR content
)

# Uncomment to submit:
# pk_vasp = ctrl.submit_job(vasp_job)
# print(f"VASP job submitted: PK={pk_vasp}")

print("VASP job prepared (submission commented out)")

## Summary

In this notebook, you learned how to:

1. **Import** the CrystalController and data models
2. **Initialize** the controller with automatic backend selection
3. **Create** CRYSTAL input files
4. **Submit** jobs using JobSubmission models
5. **Monitor** job status and view details
6. **Retrieve** job logs for debugging
7. **Support** multiple DFT codes (CRYSTAL, VASP, QE)

**Key Models:**
- `JobSubmission` - Define job parameters
- `JobStatus` - Lightweight status for lists
- `JobDetails` - Full results and diagnostics
- `DftCode` - Code selection (CRYSTAL, VASP, QUANTUM_ESPRESSO)
- `RunnerType` - Execution backend (LOCAL, SSH, SLURM, AIIDA)
- `JobState` - State enum (CREATED, SUBMITTED, RUNNING, COMPLETED, FAILED)

**Next Steps:**
- `02_materials_project.ipynb` - Fetch structures and generate VASP inputs
- `03_band_structure.ipynb` - Band structure workflows
- `04_convergence_eos.ipynb` - Convergence studies and equation of state
- `05_templates_workflows.ipynb` - Template system and phonon workflows